In [ ]:
import os
import sys
import torch

from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [57]:
import importlib
import src.train
import src.model

importlib.reload(src.train)
importlib.reload(src.model)

from src.train import sft_train_lora
from src.model import identify_target_modules

In [45]:
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [50]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 20022
})


In [49]:
target_modules = identify_target_modules(model, name_segment='self_attn')
print(target_modules)

['model.decoder.layers.0.self_attn.k_proj', 'model.decoder.layers.0.self_attn.v_proj', 'model.decoder.layers.0.self_attn.q_proj', 'model.decoder.layers.0.self_attn.out_proj', 'model.decoder.layers.1.self_attn.k_proj', 'model.decoder.layers.1.self_attn.v_proj', 'model.decoder.layers.1.self_attn.q_proj', 'model.decoder.layers.1.self_attn.out_proj', 'model.decoder.layers.2.self_attn.k_proj', 'model.decoder.layers.2.self_attn.v_proj', 'model.decoder.layers.2.self_attn.q_proj', 'model.decoder.layers.2.self_attn.out_proj', 'model.decoder.layers.3.self_attn.k_proj', 'model.decoder.layers.3.self_attn.v_proj', 'model.decoder.layers.3.self_attn.q_proj', 'model.decoder.layers.3.self_attn.out_proj', 'model.decoder.layers.4.self_attn.k_proj', 'model.decoder.layers.4.self_attn.v_proj', 'model.decoder.layers.4.self_attn.q_proj', 'model.decoder.layers.4.self_attn.out_proj', 'model.decoder.layers.5.self_attn.k_proj', 'model.decoder.layers.5.self_attn.v_proj', 'model.decoder.layers.5.self_attn.q_proj', 

In [51]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [53]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

In [58]:
sft_train_lora(
    base_model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=AutoTokenizer.from_pretrained("facebook/opt-350m"),
    adapter_name="sft_lora",
    formatting_prompts_func=formatting_prompts_func,  
    response_template=" ### Answer:",
    lora_config=lora_config,
)

/Users/cormaccureton/mambaforge/envs/reasoning-decomp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
  0%|          | 0/7509 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 8.85 GB, other allocations: 10.70 MB, max allowed: 9.07 GB). Tried to allocate 447.98 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).